## Evaluating alternative optimization solvers

So far using gurobi, which is very likely the best, fastest and most reliable solver. However, it is not open source and requires a license (in the process to obtain a price quote). We are evaluating alternative solvers to see if we can get similar results with open source solvers.

### OSQP

Solves quadratic objectives, valid for MICON's grow workflow. It requires ca 45 min to run the full modeling workflow with a community of 4 models, vs ca 10 min with gurobi. Computing time can be reduced by paralellizing MICON's code.

### SCIP

Solves MILPs so valid for cGEM reconstruction with carveME and probably SMETANA. Reconstruction of the 4 gem models required 47 min vs ca 9 min with gurobi. SCIP does not support quadratic objectives and no interface available in optlang (used by MICON). However it does support quadratic constraints.

### UPDATE: now available OSQP and HIGHS as _Mixed_ in Optlang

OSQP and HIGHS are now available as _Mixed_ in optlang. This means that they can be used to solve MILPs and QPs.

In [1]:
from micom.workflows import build
import pandas as pd

taxo_df = pd.read_csv("/home/robaina/Documents/NewAtlantis/microcom/test_nf/taxa_table.tsv", sep="\t")
manifest = build(taxonomy=taxo_df, model_db=None, out_folder="test_hybrid", cutoff=1e-2, threads=14 , solver="hybrid")

In [3]:
from micom import load_pickle

cgem_gur = load_pickle("/home/robaina/Documents/NewAtlantis/microcom/test_nf_gurobi/test_nf.pickle")
cgem_gur

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-05
Read LP format model from file /tmp/tmpxz0crmv0.lp
Reading time = 0.07 seconds
: 7675 rows, 22657 columns, 97537 nonzeros


Name,test_nf
Memory address,7f448ea41a80
Number of metabolites,7670
Number of reactions,11328
Number of genes,3097
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"e__TARA_ARC_108_MAG_00174, p__TARA_ARC_108_MAG_00174, c__TARA_ARC_108_MAG_00174, m, e__TARA_ARC_108_MAG_00083, p__TARA_ARC_108_MAG_00083, c__TARA_ARC_108_MAG_00083, p__TARA_ARC_108_MAG_00080, c__TARA_ARC_108_MAG_00080, e__TARA_ARC_108_MAG_00080, p__TARA_ARC_108_MAG_00201, c__TARA_ARC_108_MAG_00201, e__TARA_ARC_108_MAG_00201"


In [4]:
cgem_gur.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
TARA_ARC_108_MAG_00080,0.3,0.000000,2990,2013
TARA_ARC_108_MAG_00083,0.4,0.000000,2752,1825
TARA_ARC_108_MAG_00174,0.2,8.922886,3231,2085
TARA_ARC_108_MAG_00201,0.1,3.821992,1908,1300
medium,NaN,NaN,447,447


In [1]:
from micom import load_pickle

cgem_hybrid = load_pickle("/home/robaina/Documents/NewAtlantis/microcom/test_hybrid/test_nf.pickle")
cgem_hybrid

Name,test_nf
Memory address,7f4515427c70
Number of metabolites,7670
Number of reactions,11328
Number of genes,3097
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"e__TARA_ARC_108_MAG_00174, p__TARA_ARC_108_MAG_00174, c__TARA_ARC_108_MAG_00174, m, e__TARA_ARC_108_MAG_00083, p__TARA_ARC_108_MAG_00083, c__TARA_ARC_108_MAG_00083, p__TARA_ARC_108_MAG_00080, c__TARA_ARC_108_MAG_00080, e__TARA_ARC_108_MAG_00080, p__TARA_ARC_108_MAG_00201, c__TARA_ARC_108_MAG_00201, e__TARA_ARC_108_MAG_00201"


In [2]:
cgem_hybrid.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
TARA_ARC_108_MAG_00080,0.3,0.000000,2990,2013
TARA_ARC_108_MAG_00083,0.4,0.000000,2752,1825
TARA_ARC_108_MAG_00174,0.2,8.922886,3231,2085
TARA_ARC_108_MAG_00201,0.1,3.821992,1908,1300
medium,NaN,NaN,447,447


## Comparing gurobi vs OSQP for the exchanges of the cGEM

In [10]:
import pandas as pd

file_path_gurobi = 'test_nf_gurobi/exchanges.tsv'
file_path_osqp = 'test_nf_osqp/exchanges.tsv'

df_gurobi = pd.read_csv(file_path_gurobi, sep="\t")
df_osqp = pd.read_csv(file_path_osqp, sep="\t")

print(df_gurobi.shape)
print(df_osqp.shape)

(174, 9)
(947, 9)


Why so many more rows in the osqp solution? There seems to be differences in the abundace in the case of osqp, with abundances ranging from 0.1, 0.2, 0.3, how is this possible?

In [5]:
import pandas as pd

def load_and_merge_files(file_path1, file_path2, merge_columns, suffixes=('_file1', '_file2')):
    df1 = pd.read_csv(file_path1, sep='\t')
    df2 = pd.read_csv(file_path2, sep='\t')
    merged_df = pd.merge(df1, df2, on=merge_columns, suffixes=suffixes)
    return merged_df

merge_columns = ['taxon', 'reaction']
file_path_gurobi = 'test_nf_gurobi/exchanges.tsv'
file_path_osqp = 'test_nf_osqp/exchanges.tsv'

merged_df = load_and_merge_files(file_path_gurobi, file_path_osqp, merge_columns, suffixes=('_gurobi', '_osqp'))

In [6]:
merged_df


,Unnamed: 0_gurobi,taxon,sample_id_gurobi,tolerance_gurobi,reaction,flux_gurobi,abundance_gurobi,metabolite_gurobi,direction_gurobi,Unnamed: 0_osqp,sample_id_osqp,tolerance_osqp,flux_osqp,abundance_osqp,metabolite_osqp,direction_osqp
0,448,TARA_ARC_108_MAG_00174,test_nf,0.000001,EX_meoh_e,-100.000000,0.2,meoh_e,import,1180,test_nf,0.0001,-99.986546,0.2,meoh_e,import
1,458,TARA_ARC_108_MAG_00174,test_nf,0.000001,EX_thm_e,-0.000236,0.2,thm_e,import,1058,test_nf,0.0001,0.000883,0.2,thm_e,export
2,460,TARA_ARC_108_MAG_00174,test_nf,0.000001,EX_lac__L_e,-6.699874,0.2,lac__L_e,import,1165,test_nf,0.0001,-39.993122,0.2,lac__L_e,import
3,466,TARA_ARC_108_MAG_00174,test_nf,0.000001,EX_h2_e,167.740043,0.2,h2_e,export,1097,test_nf,0.0001,-50.370444,0.2,h2_e,import
4,470,TARA_ARC_108_MAG_00174,test_nf,0.000001,EX_pi_e,-1.053453,0.2,pi_e,import,1136,test_nf,0.0001,-0.910156,0.2,pi_e,import
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,1453,TARA_ARC_108_MAG_00083,test_nf,0.000001,EX_o2_e,-100.000000,0.4,o2_e,import,193,test_nf,0.0001,-23.243437,0.4,o2_e,import
168,1459,TARA_ARC_108_MAG_00083,test_nf,0.000001,EX_mg2_e,-0.007452,0.4,mg2_e,import,258,test_nf,0.0001,-0.007174,0.4,mg2_e,import
169,1464,TARA_ARC_108_MAG_00083,test_nf,0.000001,EX_dhpt_e,-0.000575,0.4,dhpt_e,import,161,test_nf,0.0001,-0.002122,0.4,dhpt_e,import
170,1466,TARA_ARC_108_MAG_00083,test_nf,0.000001,EX_h2s_e,-0.078588,0.4,h2s_e,import,27,test_nf,0.0001,0.004581,0.4,h2s_e,export


In [9]:
mismatch_count_flux = 0
mismatch_count_direction = 0

def check_flux_sign(flux1, flux2):
    return (flux1 > 0) == (flux2 > 0) or (flux1 < 0) == (flux2 < 0)

def compare_direction(dir1, dir2):
    return dir1 == dir2

# Iterate over the rows and compare the flux and direction for each method
for index, row in merged_df.iterrows():
    flux_gurobi, flux_osqp = row['flux_gurobi'], row['flux_osqp']
    direction_gurobi, direction_osqp = row['direction_gurobi'], row['direction_osqp']

    # Check for mismatch in the sign of the flux
    if not check_flux_sign(flux_gurobi, flux_osqp):
        mismatch_count_flux += 1

    # Check for mismatch in the direction
    if not compare_direction(direction_gurobi, direction_osqp):
        mismatch_count_direction += 1

# Report the results based on the merged dataframe
total_rows_merged = len(merged_df)
result_merged = {
    "Total reactions compared": total_rows_merged,
    "Mismatches in flux sign": mismatch_count_flux,
    "Mismatches in direction": mismatch_count_direction,
    "Percentage mismatches in flux sign": mismatch_count_flux / total_rows_merged * 100,
    "Percentage mismatches in direction": mismatch_count_direction / total_rows_merged * 100
}

result_merged

{'Total reactions compared': 172,
 'Mismatches in flux sign': 34,
 'Mismatches in direction': 34,
 'Percentage mismatches in flux sign': 19.767441860465116,
 'Percentage mismatches in direction': 19.767441860465116}

## Comparing elasticities calculated by gurobi and osqp

In [17]:
merge_columns = ['reaction',  'taxon', 'effector', 'direction']
file_path_gurobi = 'test_nf_gurobi/elasticities.tsv'
file_path_osqp = 'test_nf_osqp/elasticities.tsv'

merged_df = load_and_merge_files(file_path_gurobi, file_path_osqp, merge_columns, suffixes=('_gurobi', '_osqp'))

In [18]:
merged_df

,reaction,taxon,effector,direction,elasticity_gurobi,type_gurobi,elasticity_osqp,type_osqp
0,EX_h2_m,medium,EX_ca2_m,forward,-0.502101,exchanges,0.000000,exchanges
1,EX_ala_B_m,medium,EX_ca2_m,forward,0.033601,exchanges,0.000000,exchanges
2,EX_ca2_m,medium,EX_ca2_m,reverse,-0.000210,exchanges,0.000000,exchanges
3,EX_cl_m,medium,EX_ca2_m,reverse,-0.000210,exchanges,0.000000,exchanges
4,EX_co2_m,medium,EX_ca2_m,forward,-5.519588,exchanges,0.000000,exchanges
...,...,...,...,...,...,...,...,...
1011,EX_urea_m,medium,TARA_ARC_108_MAG_00201,reverse,-23.370326,abundance,1.860419,abundance
1012,EX_salchs2_m,medium,TARA_ARC_108_MAG_00201,zero,0.000000,abundance,0.881948,abundance
1013,EX_salchs2fe_m,medium,TARA_ARC_108_MAG_00201,zero,0.000000,abundance,0.881948,abundance
1014,EX_fe3mcbtt_m,medium,TARA_ARC_108_MAG_00201,zero,0.000000,abundance,0.881948,abundance


In [20]:
# Function to compare elasticity values
def compare_elasticity(merged_df):
    # Initialize a counter for mismatches and a list for differences
    mismatch_count = 0
    elasticity_differences = []

    # Iterate over the rows and compare the elasticity values
    for index, row in merged_df.iterrows():
        elasticity_file1 = row['elasticity_gurobi']
        elasticity_file2 = row['elasticity_osqp']

        # Calculate the absolute difference
        difference = abs(elasticity_file1 - elasticity_file2)
        elasticity_differences.append(difference)

        # Check for mismatches (not equal values)
        if elasticity_file1 != elasticity_file2:
            mismatch_count += 1

    # Calculate average difference
    average_difference = sum(elasticity_differences) / len(elasticity_differences)

    return mismatch_count, average_difference, elasticity_differences

# Perform the comparison
mismatch_count, average_difference, elasticity_differences = compare_elasticity(merged_df)

print(f"Total mismatches in elasticity: {mismatch_count}")
print(f"Average difference in elasticity: {average_difference}")
# Showing the first few differences for demonstration
print("First few differences in elasticity:", elasticity_differences[:5])

Total mismatches in elasticity: 695
Average difference in elasticity: 6.914775020533863
First few differences in elasticity: [0.5021007318942328, 0.033601025481158, 0.0002101838209433, 0.0002101838146462, 5.5195880484714]


## Comparing maximum growth rates calculated by gurobi and osqp

In [1]:
from micom import load_pickle

cgem_osqp = load_pickle("test_nf_osqp/test_nf.pickle")
cgem_gurobi = load_pickle("test_nf_gurobi/test_nf.pickle")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-05
Read LP format model from file /tmp/tmp11qt0me1.lp
Reading time = 0.07 seconds
: 7675 rows, 22657 columns, 97537 nonzeros


In [21]:
cgem_osqp.solver

In [14]:
cgem_osqp.cooperative_tradeoff(fraction=0.5)

,abundance,growth_rate,reactions,metabolites
compartments,,,,
TARA_ARC_108_MAG_00080,0.3,1.609761,2990,2013
TARA_ARC_108_MAG_00083,0.4,0.727060,2752,1825
TARA_ARC_108_MAG_00174,0.2,1.075930,3231,2085
TARA_ARC_108_MAG_00201,0.1,0.433339,1908,1300
medium,NaN,NaN,447,447


In [28]:
cgem_gurobi.solver

In [15]:
cgem_gurobi.cooperative_tradeoff(fraction=0.5)

,abundance,growth_rate,reactions,metabolites
compartments,,,,
TARA_ARC_108_MAG_00080,0.3,1.578236,2990,2013
TARA_ARC_108_MAG_00083,0.4,0.859071,2752,1825
TARA_ARC_108_MAG_00174,0.2,1.060397,3231,2085
TARA_ARC_108_MAG_00201,0.1,0.542107,1908,1300
medium,NaN,NaN,447,447


Max growth rates are very similar between gurobi and osqp, elasticities very different, why? numerical issues?

In [56]:
# Community objective not found in the community model, most likely created in the go when definining the optimization problem
print(cgem_gurobi.objective)

Maximize
1.0*community_objective


In [57]:
from cobra.io import  write_sbml_model

write_sbml_model(cgem_gurobi, "test_nf_gurobi/test_nf_gurobi.xml")

In [34]:
cgem_gurobi.objective = "EX_glc__D_m"
cgem_gurobi.objective_direction = "minimize"

cgem_gurobi.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
TARA_ARC_108_MAG_00080,0.3,0.0,2990,2013
TARA_ARC_108_MAG_00083,0.4,0.0,2752,1825
TARA_ARC_108_MAG_00174,0.2,0.0,3231,2085
TARA_ARC_108_MAG_00201,0.1,0.0,1908,1300
medium,NaN,NaN,447,447


### Researching on the community objective function

How is it defined? After inspecting ```print(cgem_gurobi.solver)```, apparently as a variable and contraint like this:

```text
 community_objective_equality: community_objective
   - 0.2 Growth__TARA_ARC_108_MAG_00174
   + 0.2 Growth__TARA_ARC_108_MAG_00174_reverse_e9527
   - 0.4 Growth__TARA_ARC_108_MAG_00083
   + 0.4 Growth__TARA_ARC_108_MAG_00083_reverse_f9d4e
   - 0.3 Growth__TARA_ARC_108_MAG_00080
   + 0.3 Growth__TARA_ARC_108_MAG_00080_reverse_ed394
   - 0.1 Growth__TARA_ARC_108_MAG_00201
   + 0.1 Growth__TARA_ARC_108_MAG_00201_reverse_1535b = 0
```

Which basically is just a weighted sum of all the biomass reactions of the community. The weights are the abundances of the models. The objective is to maximize the community biomass. Thus we can just include this constraint in the model when maximizing a given reaction.